In [1]:
# imports
import pandas as pd
import numpy as np

CTA Ridership - 'L' Station Entries: Monthly Day-Type Averages & Totals
https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Monthly-Day-Type-A/t2rn-p8d7

LATITUDE AND LONGITUDE INFORMATION FOR CTA STATIONS (List of 'L' Stops):
https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme

Datasets are very clean, no NaNs

In [2]:
# Read in dataset
ridership = pd.read_csv('data/CTA_-_Ridership_-__L__Station_Entries_-_Monthly_Day-Type_Averages___Totals.csv')
locations = pd.read_csv('data/CTA_-_System_Information_-_List_of__L__Stops.csv')
locations = locations[['STOP_NAME', 'STATION_NAME', 'MAP_ID', 'Location', # Only get relevant info
                       'RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']] 

In [3]:
locations.head(2)

,STOP_NAME,STATION_NAME,MAP_ID,Location,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O
0,Cicero (Loop-bound),Cicero,40420,"(41.85182, -87.745336)",False,False,False,False,False,False,False,True,False
1,Central Park (Loop-bound),Central Park,40780,"(41.853839, -87.714842)",False,False,False,False,False,False,False,True,False


In [4]:
# # See what dataframes look like
# ridership.head(2)
# locations.head(2)

Ensure that, even if MAP_ID occurs more than once, that the Location value is the same for each duplicate appearance in _locations_ dataframe

In [5]:
duplicate_map_ids = locations[locations.duplicated(subset=['MAP_ID'], keep=False)] # Duplicate MAP_IDs
# Count number of unique vals in 'Location' col for each duplicated 'MAP_ID', ensure it's == 1
consistent_locations = duplicate_map_ids.groupby('MAP_ID')['Location'].transform('nunique') == 1

# Print the rows where the condition is not met
print("Rows where MAP_ID inconsistent with Location:")
duplicate_map_ids[~consistent_locations]

Rows where MAP_ID inconsistent with Location:


,STOP_NAME,STATION_NAME,MAP_ID,Location,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O
28,Roosevelt (Howard-bound),Roosevelt,41400,"(41.867368, -87.627402)",True,False,False,False,False,False,False,False,False
108,Roosevelt (95th-bound),Roosevelt,41400,"(41.867368, -87.627402)",True,False,False,False,False,False,False,False,False
200,Roosevelt (Loop-Harlem-bound),Roosevelt,41400,"(41.867405, -87.62659)",False,False,True,False,False,False,False,False,True
292,Roosevelt (Midway-63rd-bound),Roosevelt,41400,"(41.867405, -87.62659)",False,False,True,False,False,False,False,False,True


Note: these two Locations are basically the same if you look them up, so it's safe for us to drop any rows with duplicate MAP_ID

In [6]:
locations_unique = locations.drop_duplicates(subset='MAP_ID', keep='first')

In [7]:
ridership_full = pd.merge(ridership, locations_unique[['MAP_ID', 'Location']], left_on='station_id', right_on='MAP_ID', how='left')

In [8]:
ridership_full[['latitude', 'longitude']] = ridership_full['Location'].str.strip('()').str.split(', ', expand=True)

# Convert the resulting columns to numeric type
ridership_full['latitude'] = pd.to_numeric(ridership_full['latitude'])
ridership_full['longitude'] = pd.to_numeric(ridership_full['longitude'])

In [9]:
ridership_full.head()

,station_id,stationame,month_beginning,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal,MAP_ID,Location,latitude,longitude
0,40900,Howard,01/01/2001,6233.9,3814.5,2408.6,164447,40900.0,"(42.019063, -87.672892)",42.019063,-87.672892
1,41190,Jarvis,01/01/2001,1489.1,1054.0,718.0,40567,41190.0,"(42.015876, -87.669092)",42.015876,-87.669092
2,40100,Morse,01/01/2001,4412.5,3064.5,2087.8,119772,40100.0,"(42.008362, -87.665909)",42.008362,-87.665909
3,41300,Loyola,01/01/2001,4664.5,3156.0,1952.8,125008,41300.0,"(42.001073, -87.661061)",42.001073,-87.661061
4,40760,Granville,01/01/2001,3109.8,2126.0,1453.8,84189,40760.0,"(41.993664, -87.659202)",41.993664,-87.659202


In [10]:
# List of boolean columns
boolean_columns = ['RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']
color_bools = locations.groupby('MAP_ID')[boolean_columns].any()

In [11]:
ridership_full = pd.merge(ridership_full, color_bools, left_on='station_id', right_on='MAP_ID', how='left')

In [12]:
ridership_full.head()

,station_id,stationame,month_beginning,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal,MAP_ID,Location,latitude,longitude,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O
0,40900,Howard,01/01/2001,6233.9,3814.5,2408.6,164447,40900.0,"(42.019063, -87.672892)",42.019063,-87.672892,True,False,False,False,True,True,True,False,False
1,41190,Jarvis,01/01/2001,1489.1,1054.0,718.0,40567,41190.0,"(42.015876, -87.669092)",42.015876,-87.669092,True,False,False,False,False,False,False,False,False
2,40100,Morse,01/01/2001,4412.5,3064.5,2087.8,119772,40100.0,"(42.008362, -87.665909)",42.008362,-87.665909,True,False,False,False,False,False,False,False,False
3,41300,Loyola,01/01/2001,4664.5,3156.0,1952.8,125008,41300.0,"(42.001073, -87.661061)",42.001073,-87.661061,True,False,False,False,False,False,False,False,False
4,40760,Granville,01/01/2001,3109.8,2126.0,1453.8,84189,40760.0,"(41.993664, -87.659202)",41.993664,-87.659202,True,False,False,False,False,False,False,False,False


In [13]:
# 'Washington/State', 'Madison/Wabash', 'Randolph/Wabash', 'Homan'
closed = ridership_full[ridership_full.MAP_ID.isna()]['stationame'].unique()
# Remove 4 stations that are now closed
ridership_full = ridership_full[~ridership_full['stationame'].isin(closed)]

In [16]:
# ridership_full.to_csv("ridership_with_locs.csv")